In [ ]:
#!import AutomatedAnalysis.dib

In [2]:
string path = @".\Examples\Traces\All_Conserve";
DataManager dm = DataManager.CreateAspNetData(path);
string baselineConfigName = "28c";
string comparandConfigName = "28c_con1";

## Display Benchmark Regressions

In [ ]:
Metric<BenchmarkData> avgHeapCountChanges = Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Average);
Metric<BenchmarkData> avgGCCount = Metrics.Promote(Metrics.I.TotalGCCount, Aggregation.Average);
public static Dictionary<Metric<BenchmarkData>, ExpectationDirection> HigherMemoryLowerThroughput = new()
{
    { Metrics.B.AverageMaxHeapSize, ExpectationDirection.Increase },
    { Metrics.B.AverageP50Latency, ExpectationDirection.Decrease },
    /*
    Additionally, a metric with no change is also available. The entire enum is:
    public enum ExpectationDirection
    {
        Unknown,
        Increase,
        Decrease,
        NoChange,
    }
    */
};

Dictionary< Metric<BenchmarkData>, double> thresholds = new()
{
    { Metrics.B.AverageMaxHeapSize, 5 },
    { Metrics.B.AverageP50Latency, 5 },
};

DisplayBenchmarkSummaryWithRegressions(dm, baselineConfigName, comparandConfigName, metrics: ML( Metrics.B.AverageMaxHeapSize, Metrics.B.AverageP50Latency ), benchmarkRegressionThreshold: 5, benchmarksToDisplay: 5, 
metricToleranceOverridePercentage: thresholds,
expectationDirections: HigherMemoryLowerThroughput);

## Display Iteration Anomalies

In [ ]:
// Update Average -> Safe Average.
Aggregation SafeAverage = new Aggregation((gc => {
    double safeAverage;
    
    long count = 0;
    double sum = 0;

    foreach (var item in gc)
    {
        if (item != 0 && !double.IsNaN(item))
        {
            sum += item;
            count++;
        }
    }

    safeAverage = count == 0 ? double.NaN : sum / count;
    return safeAverage;

}), "SafeAverage", "");

Metric<IterationData> avgTCPToConsider = Metrics.Promote(Metrics.G.MedianThroughputCostPercent, SafeAverage);

Dictionary<Metric<IterationData>, double> volatilityOverrides = new()
{
    { Metrics.I.Gen0MeanPauseMSec, 0.5 },
    { Metrics.I.Latency50thMS, 3 },
};

// todo: override the volatility threshold
DisplayIterationData(dm, baselineConfigName, comparandConfigName, metrics: ML( avgTCPToConsider, Metrics.I.Gen0MeanPauseMSec, Metrics.I.Latency50thMS ), benchmarkFilter: new Filter("Stage1Pgo"),
volatilityThreshold: 2, metricVolOverridePercentage: volatilityOverrides); 

## Display GC Charts

In [ ]:
ChartGCData(dm, metrics: ML( Metrics.G.MedianThroughputCostPercent, Metrics.G.NumHeaps, Metrics.G.TotalSOHStableSize ), 
 //iterationFilter: new IntFilter(1), 
 benchmarkFilter: new Filter("PlaintextMvc", "Stage1Pgo"),
 xMetric: Metrics.X.StartRelativeMSec)